<a href="https://colab.research.google.com/github/mirianbatista/vacinas/blob/master/vacinas_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!apt-get install default-jdk
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  default-jdk-headless openjdk-11-jdk
Suggested packages:
  openjdk-11-demo openjdk-11-source visualvm
The following NEW packages will be installed:
  default-jdk default-jdk-headless openjdk-11-jdk
0 upgraded, 3 newly installed, 0 to remove and 42 not upgraded.
Need to get 1,556 kB of archives.
After this operation, 1,614 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 default-jdk-headless amd64 2:1.11-68ubuntu1~18.04.1 [1,132 B]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 openjdk-11-jdk amd64 11.0.15+10-0ubuntu0.18.04.1 [1,554 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 default-jdk amd64 2:1.11-68ubunt

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

import findspark
findspark.init('spark-3.1.2-bin-hadoop3.2')

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import requests
from requests.auth import HTTPBasicAuth 
import pyspark.sql.functions as F

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
access_key = 'access_key'
secret_key = 'secret_key'

In [ ]:
spark = (SparkSession.builder.appName('Vacinas covid-19')
        .config("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
        .config("spark.hadoop.fs.s3a.fast.upload", "true")
        .config("com.amazonaws.services.s3.enableV4", "true") 
        .config('fs.s3a.access.key', access_key)
        .config('fs.s3a.secret.key', secret_key)
        .getOrCreate())

# precisa colocar na pasta jar os jars aws-java-sdk e hadoop-aws (este na mesma versão do hadoop)
# https://bigdatagurus.wordpress.com/2021/08/02/include-aws-s3-libraries-in-pyspark/

In [ ]:
r = requests.get('https://imunizacao-es.saude.gov.br/desc-imunizacao/_search', auth=('imunizacao_public', 'qlto5t&7r_@+#Tlstigi'))  
resp = r.json()

In [ ]:
rdd = spark.sparkContext.parallelize([r.text])

df = spark.read.json(rdd)

In [ ]:
rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [ ]:
df.show()

+--------------+--------------------+---------+----+
|       _shards|                hits|timed_out|took|
+--------------+--------------------+---------+----+
|{0, 0, 30, 30}|{[{84b0d93c-a207-...|    false|  12|
+--------------+--------------------+---------+----+



In [ ]:
q = df[['hits']]
q.show()

+--------------------+
|                hits|
+--------------------+
|{[{84b0d93c-a207-...|
+--------------------+



In [ ]:
z = q.withColumn("teste_source", F.col("hits")["hits"])
z.show()

+--------------------+--------------------+
|                hits|        teste_source|
+--------------------+--------------------+
|{[{84b0d93c-a207-...|[{84b0d93c-a207-4...|
+--------------------+--------------------+



In [ ]:
ss = z.withColumn('exploded_arr',explode('teste_source'))
ss.show()

+--------------------+--------------------+--------------------+
|                hits|        teste_source|        exploded_arr|
+--------------------+--------------------+--------------------+
|{[{84b0d93c-a207-...|[{84b0d93c-a207-4...|{84b0d93c-a207-4a...|
|{[{84b0d93c-a207-...|[{84b0d93c-a207-4...|{298c7793-2509-40...|
|{[{84b0d93c-a207-...|[{84b0d93c-a207-4...|{aea4b198-b849-4c...|
|{[{84b0d93c-a207-...|[{84b0d93c-a207-4...|{58b4fe2c-338d-4b...|
|{[{84b0d93c-a207-...|[{84b0d93c-a207-4...|{7e95ec35-996d-47...|
|{[{84b0d93c-a207-...|[{84b0d93c-a207-4...|{a9a13b0b-877f-41...|
|{[{84b0d93c-a207-...|[{84b0d93c-a207-4...|{72b20990-1bdb-49...|
|{[{84b0d93c-a207-...|[{84b0d93c-a207-4...|{47d5136a-5862-4c...|
|{[{84b0d93c-a207-...|[{84b0d93c-a207-4...|{45fb3ed9-3120-44...|
|{[{84b0d93c-a207-...|[{84b0d93c-a207-4...|{20e91af8-a5fe-4e...|
+--------------------+--------------------+--------------------+



In [ ]:
for f in ss.collect(): 
    print (f.exploded_arr)

Row(_id='84b0d93c-a207-4a74-a855-ca693f1593ac-i0b0', _index='desc-imunizacao-v5', _score=1.0, _source=Row(@timestamp='2022-05-26T07:59:02.209Z', @version='1', data_importacao_datalake='2022-05-26T06:07:53.000Z', data_importacao_rnds='2022-05-25T19:11:10.000Z', document_id='84b0d93c-a207-4a74-a855-ca693f1593ac-i0b0', dt_deleted=None, estabelecimento_municipio_codigo='150710', estabelecimento_municipio_nome='SAO CAETANO DE ODIVELAS', estabelecimento_razaoSocial='PREFEITURA MUNICIPAL DE SAO CAETANO DE ODIVELAS', estabelecimento_uf='PA', estabelecimento_valor='2619792', estalecimento_noFantasia='CENTRO DE SAUDE DE SAO CAETANO DE ODIVELAS', id_sistema_origem='16341', paciente_dataNascimento='2000-11-24', paciente_endereco_cep='68775', paciente_endereco_coIbgeMunicipio='150710', paciente_endereco_coPais='10', paciente_endereco_nmMunicipio='SAO CAETANO DE ODIVELAS', paciente_endereco_nmPais='BRASIL', paciente_endereco_uf='PA', paciente_enumSexoBiologico='F', paciente_id='3e1504c13445b29f8bafd

In [ ]:
df_final = ss.withColumn("source_exploded", F.col("exploded_arr")["_source"])
df_final.show()

+--------------------+--------------------+--------------------+--------------------+
|                hits|        teste_source|        exploded_arr|     source_exploded|
+--------------------+--------------------+--------------------+--------------------+
|{[{84b0d93c-a207-...|[{84b0d93c-a207-4...|{84b0d93c-a207-4a...|{2022-05-26T07:59...|
|{[{84b0d93c-a207-...|[{84b0d93c-a207-4...|{298c7793-2509-40...|{2022-05-26T07:59...|
|{[{84b0d93c-a207-...|[{84b0d93c-a207-4...|{aea4b198-b849-4c...|{2022-05-26T07:59...|
|{[{84b0d93c-a207-...|[{84b0d93c-a207-4...|{58b4fe2c-338d-4b...|{2022-05-26T07:59...|
|{[{84b0d93c-a207-...|[{84b0d93c-a207-4...|{7e95ec35-996d-47...|{2022-05-26T07:59...|
|{[{84b0d93c-a207-...|[{84b0d93c-a207-4...|{a9a13b0b-877f-41...|{2022-05-26T07:59...|
|{[{84b0d93c-a207-...|[{84b0d93c-a207-4...|{72b20990-1bdb-49...|{2022-05-26T07:59...|
|{[{84b0d93c-a207-...|[{84b0d93c-a207-4...|{47d5136a-5862-4c...|{2022-05-26T07:59...|
|{[{84b0d93c-a207-...|[{84b0d93c-a207-4...|{45fb3ed9-3

In [ ]:
for f in df_final.collect(): 
    print (f.source_exploded)

Row(@timestamp='2022-05-26T07:59:02.209Z', @version='1', data_importacao_datalake='2022-05-26T06:07:53.000Z', data_importacao_rnds='2022-05-25T19:11:10.000Z', document_id='84b0d93c-a207-4a74-a855-ca693f1593ac-i0b0', dt_deleted=None, estabelecimento_municipio_codigo='150710', estabelecimento_municipio_nome='SAO CAETANO DE ODIVELAS', estabelecimento_razaoSocial='PREFEITURA MUNICIPAL DE SAO CAETANO DE ODIVELAS', estabelecimento_uf='PA', estabelecimento_valor='2619792', estalecimento_noFantasia='CENTRO DE SAUDE DE SAO CAETANO DE ODIVELAS', id_sistema_origem='16341', paciente_dataNascimento='2000-11-24', paciente_endereco_cep='68775', paciente_endereco_coIbgeMunicipio='150710', paciente_endereco_coPais='10', paciente_endereco_nmMunicipio='SAO CAETANO DE ODIVELAS', paciente_endereco_nmPais='BRASIL', paciente_endereco_uf='PA', paciente_enumSexoBiologico='F', paciente_id='3e1504c13445b29f8bafd58065dcd42fa9c43d82e5fd019d93f34ed88a4140ef', paciente_idade=21, paciente_nacionalidade_enumNacionalid

In [ ]:
df_final.withColumn("data_importacao_datalake",df_final.source_exploded["data_importacao_datalake"]).withColumn("estabelecimento_razaoSocial",df_final.source_exploded["estabelecimento_razaoSocial"]).show()

+--------------------+--------------------+--------------------+--------------------+------------------------+---------------------------+
|                hits|        teste_source|        exploded_arr|     source_exploded|data_importacao_datalake|estabelecimento_razaoSocial|
+--------------------+--------------------+--------------------+--------------------+------------------------+---------------------------+
|{[{84b0d93c-a207-...|[{84b0d93c-a207-4...|{84b0d93c-a207-4a...|{2022-05-26T07:59...|    2022-05-26T06:07:...|       PREFEITURA MUNICI...|
|{[{84b0d93c-a207-...|[{84b0d93c-a207-4...|{298c7793-2509-40...|{2022-05-26T07:59...|    2022-05-26T06:07:...|       MUNICIPIO DE CURI...|
|{[{84b0d93c-a207-...|[{84b0d93c-a207-4...|{aea4b198-b849-4c...|{2022-05-26T07:59...|    2022-05-26T06:07:...|       PREFEITURA MUNICI...|
|{[{84b0d93c-a207-...|[{84b0d93c-a207-4...|{58b4fe2c-338d-4b...|{2022-05-26T07:59...|    2022-05-26T06:07:...|       PREFEITURA DO MUN...|
|{[{84b0d93c-a207-...|[{84b

In [ ]:
q.write.json('s3a://debug-etl-vacinas-covid/debug-etl-vacinas-covid.csv')

In [ ]:
df = spark.read.csv('s3a://debug-etl-vacinas-covid/debug-etl-vacinas-covid.csv')